In [1]:
import pandas as pd

In [2]:
import itertools

In [3]:
import glob

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
import matplotlib.pyplot as plt

In [6]:
def prepare_data(dimensions):
    
    columns = (['player index', 'co player index'] + [f'p{i}' for i in range(1, dimensions + 1)] + 
               [f'q{i}' for i in range(1, dimensions + 1)] + ['e', 'cc', 'cd', 'dc', 'dd'])
    
    files = glob.glob(f"../src/evolutionary_simulations/cooperation_data/error_0.01sdim_{dimensions}_*")
    
    dfs = [pd.read_csv(file, header=None, names=columns) for file in files]

    df = pd.concat(dfs)
    
    rhos = df[df['player index'] == df['co player index']]
    rhos['rho'] = 0
    for i, row in rhos.iterrows():
        rhos.loc[i, 'rho'] = row['cc'] + row['cd']
    df = df.merge(rhos[['player index', 'rho']], how='left', left_on='player index', right_on='player index')
    
    
    df['rho p'] = df.apply(lambda row: row['cc'] + row['cd'], axis=1)

    df['rho q'] = df.apply(lambda row: row['cc'] + row['dc'], axis=1)

    df['x'] = df.apply(lambda row: row['rho'] - row['rho p'], axis=1)

    df['y'] = df.apply(lambda row: row['rho'] - row['rho q'], axis=1)
    
    return df

In [7]:
def vaquero_method(df):
    
    sets = []

    for j, group in df.groupby('player index'):

        Q1, Q2, Q3, Q4 = [], [], [], []

        for i, row in group.iterrows():

            if row['player index'] != row['co player index']:

                if row['x'] > 0 and row['y'] > 0:
                    Q1.append(row['y'] / row['x'])

                if row['x'] < 0 and row['y'] <= 0:
                    Q2.append(row['y'] / row['x'])

                if row['x'] <= 0 and row['y'] > 0:
                    Q3.append(row['co player index'])

                if row['x'] == 0 and row['y'] == 0:

                    Q4.append(row['co player index'])

        sets.append([j, Q1, Q2, Q3, Q4])
    return sets

### One bit

In [8]:
one_bit = prepare_data(2)

In [9]:
sets = vaquero_method(one_bit)

In [10]:
pd.DataFrame(sets, columns=['player index', 'Q1 (x / y)', 'Q2 (x / y)', 'Q3 (indices)', 'Q4 (indices)'])

,player index,Q1 (x / y),Q2 (x / y),Q3 (indices),Q4 (indices)
0,1,[],[],[],[]
1,2,[],[],[1.0],[3.0]
2,3,[1.0204081632653061],[1.0204081632653061],[],[2.0]
3,4,[],[],"[1.0, 2.0, 3.0]",[]


### Two bits

In [222]:
two_bit = prepare_data(4)

In [231]:
sets = vaquero_method(two_bit)

In [232]:
pd.DataFrame(sets, columns=['player index', 'Q1 (x / y)', 'Q2 (x / y)', 'Q3 (indices)', 'Q4 (indices)'])

,player index,Q1 (x / y),Q2 (x / y),Q3 (indices),Q4 (indices)
0,1,[],[],[],[]
1,2,"[2.095451679420854, 0.9530488874492556, 0.9135...",[],"[1.0, 3.0, 5.0, 9.0]",[]
2,3,"[1.041232819593638, 1.0, 0.7928423310639319, 1...",[],[],[]
3,4,[],[],"[1.0, 2.0, 3.0, 5.0, 9.0]","[6.0, 7.0, 10.0, 11.0, 13.0]"
4,5,"[1.0412328196234282, 1.0, 1.0412123626241727]",[2882352701.513068],[],[]
5,6,[],[],"[1.0, 2.0, 3.0, 5.0, 9.0]","[4.0, 7.0, 10.0, 11.0, 13.0]"
6,7,"[1.0412328196584757, 0.6445519829973192, 1.107...",[],[],"[4.0, 6.0, 10.0, 11.0, 13.0]"
7,8,[],"[2.0954516980274027, 0.9530488885386729, 0.913...","[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 9.0]",[]
8,9,[19.503333337738876],"[1.5152617371163994, 51.89782527130969, 1.3611...",[],[]
9,10,[],"[0.6445519810383321, 1.1078496820977926, 5.490...","[1.0, 2.0, 3.0, 5.0, 9.0]","[4.0, 6.0, 7.0, 11.0, 13.0]"


### Three bits

In [225]:
three_bit = prepare_data(8)

In [233]:
sets = vaquero_method(three_bit)

In [234]:
pd.DataFrame(sets, columns=['player index', 'Q1 (x / y)', 'Q2 (x / y)', 'Q3 (indices)', 'Q4 (indices)'])

,player index,Q1 (x / y),Q2 (x / y),Q3 (indices),Q4 (indices)
0,1,[],"[5.482084341220471e+17, 5.482084341220471e+17]",[],[]
1,2,"[1.031488173529298, 0.9481764433729669, 8.9032...",[],"[1.0, 3.0, 5.0, 7.0, 9.0, 13.0, 17.0, 21.0, 33...",[]
2,3,"[2.125301306906055, 2.136551097380852, 2.11382...","[4.356168781890493, 6.726964687974346, 1.00010...",[],[]
3,4,"[0.3066187382088688, 0.3680925751198016, 1.487...",[1.0],"[1.0, 2.0, 3.0, 5.0, 6.0, 7.0, 9.0, 11.0, 13.0...",[]
4,5,"[1.0590198493906164, 1.0015455605793895, 0.828...","[8.688296993440677, 2.5254685223123556, 1.0591...",[],[]
...,...,...,...,...,...
59,60,[3.3657322888516927],[],"[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, ...",[]
60,61,"[1.0412328196584757, 0.5052665143446617, 1.102...",[0.28069164197747154],"[14.0, 25.0, 26.0, 29.0, 30.0, 36.0, 43.0, 44....","[16.0, 31.0, 46.0, 52.0]"
61,62,[],[],"[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, ...",[]
62,63,"[1.0536131689484316, 1.6724685644284785, 1.107...",[],"[10.0, 14.0, 19.0, 25.0, 26.0, 29.0, 30.0, 42....",[]


### Four bits

In [235]:
four_bit = prepare_data(16)

In [237]:
sets = vaquero_method(four_bit)

In [238]:
pd.DataFrame(sets, columns=['player index', 'Q1 (x / y)', 'Q2 (x / y)', 'Q3 (indices)', 'Q4 (indices)'])

,player index,Q1 (x / y),Q2 (x / y),Q3 (indices),Q4 (indices)
0,1,[],"[470692250961.0178, 4754467181.919373, 4754467...",[],[]
1,2,"[1.1778261084563888, 0.987750039635907, 1.0413...",[],"[1.0, 3.0, 5.0, 7.0, 9.0, 13.0, 17.0, 19.0, 21...",[]
2,3,"[2.147340278653103, 2.1351953799526076, 2.1356...","[5.693177909845503, 8.232698998023187, 5.76735...",[],[]
3,4,"[6.741401512575014, 5.228379702432307, 1.27581...","[1.318365635034073, 0.20580432133294754, 6.321...","[1.0, 2.0, 3.0, 5.0, 7.0, 9.0, 10.0, 11.0, 13....",[]
4,5,"[1.0847249330974662, 1.0866014890150115, 0.760...","[1.0830785947030641, 0.9748031769471045, 4.328...",[],[]
...,...,...,...,...,...
251,252,"[0.7495992629571121, 0.6509571590183887, 0.202...",[],"[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, ...",[]
252,253,"[1.0422339917585197, 0.5300832684703896, 1.081...","[0.7586565693241132, 3.6163754390285665]","[68.0, 72.0, 84.0, 88.0, 100.0, 104.0, 114.0, ...",[]
253,254,"[20.998420667036843, 5.155216507855901, 0.1346...",[],"[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, ...",[]
254,255,"[1.0712280116121697, 2.935843257349387, 1.1180...","[0.045535629631219536, 4.687307716367075, 3.38...","[14.0, 26.0, 30.0, 46.0, 58.0, 60.0, 62.0, 78....",[]
